In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import pandas as pd

In [ ]:
adata_X = np.load("tmp/X.npy")
adata = sc.AnnData(
    X=adata_X,
    obs=pd.DataFrame(index=[str(i) for i in range(len(adata_X))]),
    var=pd.DataFrame(index=np.load("tmp/var.npy", allow_pickle=True)),
)
adata.obs["batch"] = np.load("tmp/batch.npy", allow_pickle=True)
adata.obs["x"] = np.load("tmp/x_coord.npy", allow_pickle=True)
adata.obs["y"] = np.load("tmp/y_coord.npy", allow_pickle=True)
adata.obs["Immunocentric_Type_With_P14"] = np.load("tmp/type.npy", allow_pickle=True)

In [ ]:
adata_sub = adata[
    adata.obs.batch.isin(["human_05_r1", "human_05_r2", "human_09_r1", "human_09_r2"])
]

In [ ]:
zeros = np.zeros(len(adata_sub.obs))
zeros[np.where(adata_sub.obs.batch.isin(["human_09_r2"]))[0]] = (
    adata_sub.obs["x"][np.where(adata_sub.obs.batch.isin(["human_09_r2"]))[0]] + 300000
)
zeros[np.where(adata_sub.obs.batch.isin(["human_05_r2"]))[0]] = (
    adata_sub.obs["x"][np.where(adata_sub.obs.batch.isin(["human_05_r2"]))[0]] + 200000
)
zeros[np.where(adata_sub.obs.batch.isin(["human_05_r1"]))[0]] = (
    adata_sub.obs["x"][np.where(adata_sub.obs.batch.isin(["human_05_r1"]))[0]] + 100000
)
zeros[np.where(adata_sub.obs.batch.isin(["human_09_r1"]))[0]] = adata_sub.obs["x"][
    np.where(adata_sub.obs.batch.isin(["human_09_r1"]))[0]
]

In [ ]:
adata_sub.obs["x"] = zeros

In [ ]:
adata_sub.obsm["X_spatial"] = adata_sub.obs[["x", "y"]].values

In [ ]:
import numpy as np
from scipy.spatial import cKDTree
import matplotlib.pyplot as plt

# Assuming that `adata_sub` is your Seurat object and you want to compute distances on `X_spatial`
spatial_coords = adata_sub.obsm["X_spatial"]

# Create a KD-Tree from the spatial coordinates
kdtree = cKDTree(spatial_coords)

# Query the KD-Tree to find the distances to the nearest neighbors
min_distances, _ = kdtree.query(
    spatial_coords, k=2
)  # k=2 to find the two nearest neighbors (the first one is the point itself)

# Extract the distances to the second nearest neighbor (the first one is the point itself)
min_distances = min_distances[:, 1]

# Plot a histogram
plt.hist(min_distances, bins=1000, edgecolor="k")
plt.xlabel("Distance to Nearest Neighbor")
plt.ylabel("Frequency")
plt.title("Histogram of Distance to Nearest Neighbor (using KD-Tree)")
plt.show()

In [ ]:
np.percentile(min_distances, 30)

In [ ]:
adata_sub.write_h5ad("tmp/cellchat_preparation_human_signature.h5")

In [ ]:
np.unique(adata_sub.obs["Immunocentric_Type_With_P14"])